In [3]:
# import
import aiohttp
import pkuseg
import pymongo
import zhconv
import joblib
from types import TracebackType
from typing import Dict, Optional, Type
from aip import AipNlp
from motor import motor_asyncio
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError

    
# CONFIG
if platform.system() == 'Windows':
    # CODE_PATH
    CODE_PATH = os.path.join(os.environ['onedrive'], 'STN/10-FirstYear/code')
    DATA_PATH = os.path.join(os.environ['onedrive'], 'STN/10-FirstYear/data')
    BERT_PATH = os.path.join(os.environ['onedrive'], 'Data/BERT')
    ERNIE_PATH = os.path.join(os.environ['onedrive'], 'Data/baidu-paddle/LARK/ERNIE')
    %cd {CODE_PATH}
elif platform.system() == 'Linux':
    # CODE_PATH
    CODE_PATH = '/mnt/c/Users/rossz/OneDrive/STN/10-FirstYear/code'
    DATA_PATH = '/mnt/c/Users/rossz/OneDrive/STN/10-FirstYear/data'
    %cd {CODE_PATH}

# Baidu NLP
def get_token(accounts):
    '''get API tokens'''
    url = f"https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials"
    for account in accounts:
        # client_id 为官网获取的AK， client_secret 为官网获取的SK
        param = {'client_id': account['apiKey'],
                 'client_secret': account['secretKey']}
        resp = requests.get(url, param).json()
        yield resp['access_token']
        
accounts = [{'appId': '16105374', # ross.zhu@outlook.com
            'apiKey': 'zShobECEYX2B2CNrcz3QsM3I',
            'secretKey': 'xM8g1lkhTd8b7DYGv2H9j7g2hvthPGE5'},
           {'appId': '16218655', # 18868819096
            'apiKey': 'AG2ygGZbCPqDW25ZC6FinQRy',
            'secretKey': 'pcx8pFwjVhD0oGDsdm58bzt7Zm9zFpWk'},
           {'appId': '16226200', # 13566633705
            'apiKey': 'ymmGiPQoAoSF2KhcGVKNZEVj',
            'secretKey': '78oQXsRtdmRcINEGtP3aP2y4sKA7HerU'},
           {'appId': '16226243', # 13606586786
            'apiKey': 'jIy3T25rYR1Kj9BlCKPcHbYe',
            'secretKey': 'jORNEeu7Qnl6L1Hr07NomigmsUtXlloL'},
           {'appId': '16230746', # 李锐
            'apiKey': 't6sWsoIYinoGBgwbLiCopQHR',
            'secretKey': 'evq4UygxQ555LXzVzaDWQS2TfGNUHrYf'},
           {'appId': '16307961', # 俞嘉炜
            'apiKey': 'TKIEhGe1fIY89VlYm8yjuDZW',
            'secretKey': '6kDBxZnGZEsK4y9ntg94VOBSIjLS0UQs'},
           {'appId': '16305932', # 芳菲
            'apiKey': 'Y7NjBe2sqhfIjSIaB51wvlHH',
            'secretKey': 'lqLW2XeSj0xBvCbUASh0E2wuMmW6e4ox'},
           {'appId': '16305931', # 小毫生
            'apiKey': 'D2GGEWS80KsU9mtft1WnuR0N',
            'secretKey': '8FbvLeGD5aXLL6bpKSEZoG7CRt1Q9oqo'},
           {'appId': '16306399', # 忻小宇
            'apiKey': 'E89qU1OMrwvyanH7xixkFVrR',
            'secretKey': 'ljPLgCtzOKLZuG56DPX9hvrVFHRsWHqf'},
           {'appId': '16306475', # 杨老板
            'apiKey': 'yehP04KXoWADTGUyLF8WK0hN',
            'secretKey': 'jfp3wxqupbWfEoNi7L316LdfbUmDmRUo'},
           ]
tokens = list(get_token(accounts))
baidu_clients = [AipNlp(**account) for account in accounts] 

# MongoDB-pymongo
pymongo_client = MongoClient('localhost', 27018)
db = pymongo_client.exchange
coll_embedding_bd = db.embedding_bd
coll_vocab_without_emb = db.vocab_without_emb
coll_vocab_to_request = db.vocab_to_request
coll_vocab_pos = db.vocab_pos
coll_vocab_ne = db.vocab_ne
coll_text_seg_bd = db.text_seg_bd
coll_title_seg_bd = db.title_seg_bd
coll_test = db.test

C:\Users\rossz\OneDrive\STN\10-FirstYear\code


In [2]:
x = ld('cmt_title_segged_pku')

In [7]:
x = joblib.load('../data/cmt_text_cls_emb_0.joblib')
len(x)

2000000